In [1]:
import tensorflow as tf 
from keras.preprocessing import sequence
import keras 
import os
import numpy as np

In [2]:
#Get data
path_to_file = "./SongLyrics.txt"
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [9]:
#Encode the text
vocab = sorted(set(text))

char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [10]:
print(f'Text: {text[:200]}\nEncoded: {text_as_int[:200]}')

Text: 

The state of Man
And of machine
Streets of Japan
2517
2517

The fate of Man
And of machine
It all began
2517
2517



Salvation code
The salvation code
The salvation code

I hol
Encoded: [ 1  0  1  1  0 38 53 50  2 64 65 46 65 50  2 60 51  2 32 46 59  1  0 20
 59 49  2 60 51  2 58 46 48 53 54 59 50  1  0 37 65 63 50 50 65 64  2 60
 51  2 29 46 61 46 59  1  0 13 14 12 15  1  0 13 14 12 15  1  0  1  0 38
 53 50  2 51 46 65 50  2 60 51  2 32 46 59  1  0 20 59 49  2 60 51  2 58
 46 48 53 54 59 50  1  0 28 65  2 46 57 57  2 47 50 52 46 59  1  0 13 14
 12 15  1  0 13 14 12 15  1  0  1  0  1  0  1  1  0 37 46 57 67 46 65 54
 60 59  2 48 60 49 50  1  0 38 53 50  2 64 46 57 67 46 65 54 60 59  2 48
 60 49 50  1  0 38 53 50  2 64 46 57 67 46 65 54 60 59  2 48 60 49 50  1
  0  1  0 28  2 53 60 57]


In [7]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        print('Except: Already a numpy array\n')
    return ''.join(idx2char[ints])

print(int_to_text(text_as_int))

Except: Already a numpy array



The state of Man
And of machine
Streets of Japan
2517
2517

The fate of Man
And of machine
It all began
2517
2517



Salvation code
The salvation code
The salvation code

I hol


In [12]:
#Creating Training samples
seq_lenght = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_lenght+1,drop_remainder = True)

In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

def build_model(vocab_size, embeding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embeding_dim, batch_input_shape=[batch_size,None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model= build_model(VOCAB_SIZE,EMBEDING_DIM,RNN_UNITS,BATCH_SIZE)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           18432     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 72)            73800     
                                                                 
Total params: 5,339,208
Trainable params: 5,339,208
Non-trainable params: 0
_________________________________________________________________


In [15]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [41]:
#LoadModel from lastest Checkpoint to train
model= build_model(VOCAB_SIZE,EMBEDING_DIM,RNN_UNITS,64)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1,None]))

In [42]:
model.compile(optimizer='adam', loss=loss)

In [43]:
checkpoint_dir = './LyricGeneratorCheckpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_freq=50
)


history = model.fit(data, epochs=250, callbacks=[checkpoint_callback])

Epoch 1/250
7/7 [==============================] - 2s 159ms/step - loss: 0.2082
Epoch 2/250
7/7 [==============================] - 1s 154ms/step - loss: 0.1071
Epoch 3/250
7/7 [==============================] - 1s 155ms/step - loss: 0.1046
Epoch 4/250
7/7 [==============================] - 1s 154ms/step - loss: 0.1009
Epoch 5/250
7/7 [==============================] - 1s 154ms/step - loss: 0.1024
Epoch 6/250
7/7 [==============================] - 1s 154ms/step - loss: 0.1025
Epoch 7/250
7/7 [==============================] - 1s 153ms/step - loss: 0.1007
Epoch 8/250
7/7 [==============================] - 2s 155ms/step - loss: 0.1007
Epoch 9/250
7/7 [==============================] - 1s 154ms/step - loss: 0.1008
Epoch 10/250
7/7 [==============================] - 1s 154ms/step - loss: 0.1007
Epoch 11/250
7/7 [==============================] - 1s 154ms/step - loss: 0.0981
Epoch 12/250
7/7 [==============================] - 1s 154ms/step - loss: 0.1028
Epoch 13/250
7/7 [===================

In [44]:
#LoadModel from lastest Checkpoint to use

model= build_model(VOCAB_SIZE,EMBEDING_DIM,RNN_UNITS,1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1,None]))

In [45]:
def generate_text(model, start_string, num_generate=400, temperature=1):
    
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval,0)
    
    text_generated = []
    
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions,0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id] , 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return start_string + ''.join(text_generated)

In [ ]:
start_str = str(input('Type a starting string: '))

lenght = input('Type song lenght, leave blank for default(400): ')
    
if not lenght or lenght == "":
    lenght = 400
else:
    lenght = int(lenght)
    
temp = input('Type temperature, leave blank for default(1.0): ')
    
if not temp or temp == "":
    temp = 1.0
else:
    temp = float(temp)

print(generate_text(model, start_str, lenght, temp))

In [52]:
print(generate_text(model,"monke",1000,1.3))

monkess is coming
'Cause the writing's on the wall
We talk in thet thought practions ald sust between
The Darkness and the Liknow what to do so I keep praying for theee when you're next to me
You are my Aphelion

I dorld through the eyou swith E.E.V. next to God and sinners reconciled
Joyful, alling in between
Darkness and the Light



R-, red, red pray name as the Heavens crlear
Andrise

You are Phoenix
You are Phoenix

Halo!
God fld tancodream
In monochrome
You and I will ride the afterglow

(You and, you and me)
(Yoh one the faces of the clones
Decrecreak toll
Or our own, we face what is coming
'Cause the writing's on the wall

Fade into this waking dream
As darkness falls
On our own, it's you and me, baby
When the writing's on the wall
Lost inside the datastream
Under nightfall
On our own, it's you and mand
Edeng for your life inside a killer thriller
Tonight

You hear the door slam
And realize there's nowhere left to run (ow!)
You feel the cold
